In [1]:
import pandas as pd
from InvestingCalendarPreprocessor import InvestingCalendarPreprocessor
preprocessor = InvestingCalendarPreprocessor()

## Indicador's exploration

In [2]:
preprocessor.explore_indicators(year=2023, thres=10)

18440 indicators read
CPI (YoY) 642 , 46
CPI (MoM) 562 , 42
Trade Balance 441 , 36
Manufacturing PMI 438 , 30
Unemployment Rate 433 , 38
Retail Sales (YoY) 357 , 29
Industrial Production (YoY) 292 , 26
GDP (YoY) 285 , 41
PPI (YoY) 280 , 25
GDP (QoQ) 246 , 35
Services PMI 240 , 13
Consumer Confidence 228 , 19
PPI (MoM) 198 , 16
Retail Sales (MoM) 189 , 15
Composite PMI 184 , 10
Industrial Production (MoM) 176 , 14
Interest Rate Decision 175 , 22
Thomson Reuters IPSOS PCSI (MoM) 168 , 14
Core CPI (YoY) 162 , 13
Current Account 112 , 14
Car Registration (YoY) 100 , 10
Car Registration (MoM) 90 , 10
M3 Money Supply 89 , 11


In [3]:
df_grouped = preprocessor.get_count_by_country_and_month()
df_grouped.to_csv('count_by_country_and_month.csv')
df_grouped

14863 indicators read


,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,Poland,Chile,Greece,Peru,Hungary,Czechia,Egypt,Colombia,Argentina,Russia
1999-01-01,107,25,40,28,11,10,26,19,6,4,...,8,3,3,2,6,6,3,6,3,6
1999-02-01,95,24,40,28,11,10,26,19,6,4,...,8,3,3,2,8,6,3,6,3,6
1999-03-01,109,24,40,28,12,10,26,19,6,4,...,8,3,3,2,8,6,3,6,3,6
1999-04-01,109,25,40,28,12,10,26,19,6,4,...,8,3,3,2,8,6,3,6,3,6
1999-05-01,107,24,40,29,12,10,26,19,6,4,...,8,3,3,2,8,6,3,6,3,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,370,83,87,46,51,42,68,46,14,15,...,22,13,9,7,18,21,6,11,10,30
2023-09-01,354,82,87,56,48,42,61,44,14,19,...,19,12,11,5,19,23,6,10,14,24
2023-10-01,343,86,80,47,51,38,60,58,12,12,...,19,12,8,4,17,20,5,10,9,18
2023-11-01,353,77,89,50,53,26,67,46,14,16,...,23,13,10,7,18,19,6,11,9,29


## GDP

In [4]:
gdp_qoq = preprocessor.process_quaterly_indicator('GDP (QoQ)', 'GDP Annualized (QoQ)')
gdp_qoq

3201 indicators read
Missing countries
['Taiwan', 'India', 'Saudi Arabia', 'Malaysia', 'United Arab Emirates', 'Kuwait', 'Turkiye', 'Chile', 'Peru', 'Hungary', 'Egypt', 'Argentina', 'Russia']
Data coverage since 1999 (50 countries): 36.2 %
Data coverage since 2013 (30 countries): 76.4 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
2007-10-01,2008-02-13 18:50:00,Japan,GDP (QoQ),GDP (QoQ),2007-10-01 00:00:00,0.0
2007-10-01,2008-02-13 21:00:00,Germany,GDP (QoQ),German GDP (QoQ),2007-10-01 00:00:00,0.3
2007-10-01,2008-02-13 21:45:00,France,GDP (QoQ),French GDP (QoQ),2007-10-01 00:00:00,0.3
2007-10-01,2008-02-14 00:00:00,Eurozone,GDP (QoQ),GDP (QoQ),2007-10-01 00:00:00,0.0
2007-10-01,2008-02-26 02:00:00,Germany,GDP (QoQ),German GDP (QoQ),2007-10-01 00:00:00,0.3
...,...,...,...,...,...,...
2023-07-01,2023-12-22 02:00:00,United Kingdom,GDP (QoQ),GDP (QoQ) (Q3),2023-07-01 00:00:00,-0.1
2023-07-01,2023-12-22 02:00:00,Denmark,GDP (QoQ),GDP (QoQ) (Q3),2023-07-01 00:00:00,-0.7
2023-07-01,2023-12-22 03:00:00,Spain,GDP (QoQ),Spanish GDP (QoQ) (Q3),2023-07-01 00:00:00,0.3


In [5]:
df = preprocessor.transform_to_countries_df(gdp_qoq, freq='Q')
df.dropna(thresh=5, axis=1)

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Netherlands,Denmark,...,South Africa,Mexico,Thailand,Indonesia,Qatar,Philippines,Poland,Greece,Czechia,Colombia
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-01,2.6,0.0,0.1,0.0,0.1,0.0,-0.4,0.5,0.6,0.9,...,-1.3,0.5,-1.5,0.36,2.7,2.4,-2.4,1.4,-0.4,0.7
2023-01-01,2.0,0.7,0.1,3.1,0.2,0.3,-0.3,0.2,-0.3,0.6,...,0.4,1.0,1.9,-0.92,-3.9,1.1,3.8,-0.1,0.0,1.4
2023-04-01,2.1,1.2,0.2,-0.2,0.5,0.0,0.0,0.4,-0.2,-0.3,...,0.6,0.8,0.2,3.86,0.5,-0.9,-2.2,1.3,0.0,-1.0
2023-07-01,4.9,-0.7,-0.1,-1.1,-0.1,0.3,-0.1,0.2,-0.3,-0.7,...,-0.2,1.1,0.8,1.6,NaN,3.3,1.5,0.0,-0.6,0.2


In [6]:
preprocessor.process_quaterly_indicator('GDP (YoY)', 'GDP Quarterly (YoY)')

3255 indicators read
Missing countries
['United States', 'Belgium', 'Norway', 'Israel', 'Austria', 'Kuwait', 'Egypt']
Data coverage since 1999 (50 countries): 39.6 %
Data coverage since 2013 (30 countries): 74.7 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,Finland,GDP (YoY),Finnish GDP (YoY) (Dec),1998-12-01 00:00:00,3.49
1999-01-01,1999-02-01 04:00:00,Finland,GDP (YoY),Finnish GDP (YoY) (Jan),1999-01-01 00:00:00,6.92
1999-02-01,1999-03-01 04:00:00,Finland,GDP (YoY),Finnish GDP (YoY) (Feb),1999-02-01 00:00:00,6.22
1999-03-01,1999-04-01 04:00:00,Finland,GDP (YoY),Finnish GDP (YoY) (Mar),1999-03-01 00:00:00,7.40
1999-04-01,1999-05-01 04:00:00,Finland,GDP (YoY),Finnish GDP (YoY) (Apr),1999-04-01 00:00:00,4.30
...,...,...,...,...,...,...
2023-07-01,2023-12-22 02:00:00,Denmark,GDP (YoY),GDP (YoY) (Q3),2023-07-01 00:00:00,-0.30
2023-07-01,2023-12-22 03:00:00,Spain,GDP (YoY),Spanish GDP (YoY) (Q3),2023-07-01 00:00:00,1.80
2023-10-01,2024-01-01 19:00:00,Singapore,GDP (YoY),GDP (YoY) (Q4),2023-10-01 00:00:00,2.80


## Labour

In [7]:
preprocessor.process_monthly_indicator('Unemployment Rate')

10018 indicators read
Missing countries
['Belgium', 'Ireland', 'India', 'Saudi Arabia', 'Thailand', 'Indonesia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye', 'Philippines', 'Egypt']
Data coverage since 1999 (50 countries): 53.8 %
Data coverage since 2013 (30 countries): 65.9 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,United Kingdom,Unemployment Rate,Unemployment Rate (Dec),1998-12-01 00:00:00,6.1
1998-12-01,1999-01-01 04:00:00,Canada,Unemployment Rate,Unemployment Rate (Dec),1998-12-01 00:00:00,8.1
1998-12-01,1999-01-01 04:00:00,Switzerland,Unemployment Rate,Unemployment Rate s.a. (Dec),1998-12-01 00:00:00,3.2
1998-12-01,1999-01-01 04:00:00,Germany,Unemployment Rate,German Unemployment Rate (Dec),1998-12-01 00:00:00,10.8
1998-12-01,1999-01-01 04:00:00,Australia,Unemployment Rate,Unemployment Rate (Dec),1998-12-01 00:00:00,7.2
...,...,...,...,...,...,...
2023-12-01,2024-01-09 03:00:00,Czechia,Unemployment Rate,Unemployment Rate (Dec),2023-12-01 00:00:00,3.7
2024-11-01,2024-01-09 04:00:00,Italy,Unemployment Rate,Italian Monthly Unemployment Rate (Nov),2024-11-01 00:00:00,7.5
2024-11-01,2024-01-09 05:00:00,Eurozone,Unemployment Rate,Unemployment Rate (Nov),2024-11-01 00:00:00,6.4


## Prices

In [8]:
cpi_mom = preprocessor.process_monthly_indicator('CPI (MoM)')
cpi_mom

12565 indicators read
Missing countries
['Australia', 'New Zealand', 'India', 'Thailand', 'Indonesia', 'Greece', 'Egypt']
Data coverage since 1999 (50 countries): 61.8 %
Data coverage since 2013 (30 countries): 80.7 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,United Kingdom,CPI (MoM),CPI (MoM) (Dec),1998-12-01 00:00:00,0.30
1998-12-01,1999-01-01 04:00:00,United States,CPI (MoM),CPI (MoM) (Dec),1998-12-01 00:00:00,0.20
1998-12-01,1999-01-01 04:00:00,Canada,CPI (MoM),CPI (MoM) (Dec),1998-12-01 00:00:00,-0.30
1998-12-01,1999-01-01 04:00:00,Mexico,CPI (MoM),CPI (MoM) (Dec),1998-12-01 00:00:00,2.44
1998-12-01,1999-01-01 04:00:00,Sweden,CPI (MoM),CPI (MoM) (Dec),1998-12-01 00:00:00,-0.20
...,...,...,...,...,...,...
2023-12-01,2024-01-11 20:30:00,China,CPI (MoM),CPI (MoM) (Dec),2023-12-01 00:00:00,0.10
2023-12-01,2024-01-12 02:30:00,Hungary,CPI (MoM),CPI (MoM) (Dec),2023-12-01 00:00:00,-0.30
2023-12-01,2024-01-12 02:45:00,France,CPI (MoM),French CPI (MoM) (Dec),2023-12-01 00:00:00,0.10


In [9]:
preprocessor.transform_to_countries_df(cpi_mom, freq='M').dropna(thresh=5, axis=1)

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Netherlands,Denmark,Sweden,...,Turkiye,Philippines,Poland,Chile,Peru,Hungary,Czechia,Colombia,Argentina,Russia
1999-01-01,0.2,NaN,-0.6,0.2,-0.4,0.2,-0.1,NaN,NaN,0.0,...,NaN,1.85,1.5,-0.33,0.01,NaN,0.7,2.21,NaN,8.3
1999-02-01,0.2,NaN,0.2,0.1,0.3,0.3,0.1,NaN,NaN,0.0,...,NaN,0.14,0.6,0.07,0.31,NaN,0.0,1.7,NaN,4.2
1999-03-01,0.0,NaN,0.5,0.4,0.4,0.1,0.0,NaN,NaN,0.4,...,NaN,-0.28,1.0,0.64,0.61,NaN,0.0,0.94,NaN,2.8
1999-04-01,0.1,NaN,0.4,0.5,0.3,0.2,0.5,NaN,NaN,0.2,...,NaN,0.0,0.8,0.38,0.59,NaN,0.1,0.78,NaN,3.0
1999-05-01,0.7,NaN,0.3,0.2,0.0,-0.2,0.0,NaN,NaN,0.2,...,NaN,0.0,0.7,0.12,0.47,NaN,0.0,0.48,NaN,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,0.6,0.2,0.3,0.4,1.0,0.2,0.3,0.4,-0.7,0.1,...,9.09,1.1,0.0,0.1,0.38,0.7,0.2,0.7,12.3,0.3
2023-09-01,0.4,0.3,0.5,-0.1,-0.5,-0.1,0.3,-0.4,-0.3,0.5,...,4.75,1.1,-0.4,0.7,0.02,0.4,-0.7,0.54,12.7,0.9
2023-10-01,0.0,NaN,0.0,0.1,0.1,0.1,0.0,0.4,0.3,0.2,...,3.43,-0.2,0.3,0.4,-0.32,-0.1,0.1,0.25,8.3,0.8
2023-11-01,0.1,NaN,-0.2,0.1,-0.2,-0.2,-0.4,-1.0,-0.3,0.3,...,3.28,0.2,0.7,0.7,-0.16,0.0,0.1,0.47,12.9,1.1


In [10]:
preprocessor.process_monthly_indicator('CPI (YoY)')

17378 indicators read
Missing countries
['Indonesia', 'Chile', 'Peru']
Data coverage since 1999 (50 countries): 80.0 %
Data coverage since 2013 (30 countries): 86.5 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,United Kingdom,CPI (YoY),CPI (YoY) (Dec),1998-12-01 00:00:00,1.60
1998-12-01,1999-01-01 04:00:00,United States,CPI (YoY),CPI (YoY) (Dec),1998-12-01 00:00:00,1.60
1998-12-01,1999-01-01 04:00:00,Canada,CPI (YoY),CPI (YoY) (Dec),1998-12-01 00:00:00,1.00
1998-12-01,1999-01-01 04:00:00,Mexico,CPI (YoY),CPI (YoY) (Dec),1998-12-01 00:00:00,18.61
1998-12-01,1999-01-01 04:00:00,Sweden,CPI (YoY),CPI (YoY) (Dec),1998-12-01 00:00:00,-1.10
...,...,...,...,...,...,...
2023-12-01,2024-01-12 02:45:00,France,CPI (YoY),French CPI (YoY),2023-12-01 00:00:00,3.70
2023-12-01,2024-01-12 03:00:00,Spain,CPI (YoY),Spanish CPI (YoY) (Dec),2023-12-01 00:00:00,3.10
2023-12-01,2024-01-12 05:00:00,Greece,CPI (YoY),Greek CPI (YoY) (Dec),2023-12-01 00:00:00,3.50


In [11]:
preprocessor.process_monthly_indicator('PPI (MoM)')

4530 indicators read
Missing countries
['United Kingdom', 'Canada', 'Australia', 'Netherlands', 'Denmark', 'Spain', 'Hong Kong', 'Singapore', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Portugal', 'China', 'Taiwan', 'India', 'Saudi Arabia', 'Thailand', 'Indonesia', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary', 'Egypt', 'Colombia', 'Argentina']
Data coverage since 1999 (50 countries): 26.2 %
Data coverage since 2013 (30 countries): 40.5 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,Sweden,PPI (MoM),PPI (MoM) (Dec),1998-12-01 00:00:00,0.00
1998-12-01,1999-01-01 04:00:00,Italy,PPI (MoM),Italian PPI (MoM) (Dec),1998-12-01 00:00:00,-0.20
1998-12-01,1999-01-01 04:00:00,Korea,PPI (MoM),PPI (MoM) (Dec),1998-12-01 00:00:00,-0.60
1998-12-01,1999-01-01 04:00:00,Switzerland,PPI (MoM),PPI (MoM) (Dec),1998-12-01 00:00:00,-0.20
1998-12-01,1999-01-01 04:00:00,Germany,PPI (MoM),German PPI (MoM) (Dec),1998-12-01 00:00:00,-0.10
...,...,...,...,...,...,...
2023-12-01,2024-01-03 02:00:00,Turkiye,PPI (MoM),PPI (MoM) (Dec),2023-12-01 00:00:00,1.14
2024-11-01,2024-01-04 07:00:00,Brazil,PPI (MoM),Brazilian PPI (MoM) (Nov),2024-11-01 00:00:00,-0.43
2024-11-01,2024-01-05 05:00:00,Eurozone,PPI (MoM),PPI (MoM) (Nov),2024-11-01 00:00:00,-0.30


In [4]:
ppi_yoy = preprocessor.process_monthly_indicator('PPI (YoY)')
ppi_yoy

6030 indicators read
Missing countries
['United Kingdom', 'Canada', 'Netherlands', 'Denmark', 'Singapore', 'Belgium', 'Israel', 'Ireland', 'New Zealand', 'Portugal', 'Taiwan', 'India', 'Brazil', 'Saudi Arabia', 'Thailand', 'Indonesia', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Chile', 'Peru', 'Egypt', 'Colombia', 'Argentina']
Data coverage since 1999 (50 countries): 35.7 %
Data coverage since 2013 (30 countries): 49.9 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,Sweden,PPI (YoY),PPI (YoY) (Dec),1998-12-01 00:00:00,-1.73
1998-12-01,1999-01-01 04:00:00,Italy,PPI (YoY),Italian PPI (YoY) (Dec),1998-12-01 00:00:00,-1.40
1998-12-01,1999-01-01 04:00:00,Korea,PPI (YoY),PPI (YoY) (Dec),1998-12-01 00:00:00,3.60
1998-12-01,1999-01-01 04:00:00,Switzerland,PPI (YoY),PPI (YoY) (Dec),1998-12-01 00:00:00,-3.10
1998-12-01,1999-01-01 04:00:00,Germany,PPI (YoY),German PPI (YoY) (Dec),1998-12-01 00:00:00,-2.00
...,...,...,...,...,...,...
2024-03-01,2024-04-30 02:45:00,France,PPI (YoY),PPI (YoY) (Mar),2024-03-01 00:00:00,-7.50
2024-03-01,2024-04-30 03:00:00,Austria,PPI (YoY),Austrian PPI (YoY) (Mar),2024-03-01 00:00:00,-6.30
2024-03-01,2024-04-30 05:00:00,Greece,PPI (YoY),Greek PPI (YoY) (Mar),2024-03-01 00:00:00,-0.10


In [5]:
df = preprocessor.transform_to_countries_df(ppi_yoy, freq='M')
df.dropna(thresh=5, axis=1)

,United States,Japan,France,Switzerland,Germany,Australia,Sweden,Spain,Italy,Finland,...,Korea,South Africa,Mexico,Turkiye,Philippines,Poland,Greece,Hungary,Czechia,Russia
1999-01-01,NaN,-2.4,NaN,-3.0,-2.3,NaN,-2.48,NaN,-1.7,-5.51,...,-1.7,NaN,NaN,49.99,NaN,3.9,NaN,NaN,0.8,NaN
1999-02-01,NaN,-2.2,NaN,-2.9,-2.5,NaN,-3.57,NaN,-1.9,-5.71,...,-4.4,NaN,NaN,48.28,NaN,3.7,NaN,NaN,0.0,NaN
1999-03-01,NaN,-1.9,NaN,-2.7,-2.3,NaN,-3.15,NaN,-1.9,-5.42,...,-4.3,NaN,NaN,48.23,NaN,4.7,NaN,NaN,0.1,NaN
1999-04-01,NaN,-1.9,NaN,-2.7,-1.7,NaN,-2.62,NaN,-1.7,-4.19,...,-4.1,NaN,NaN,50.05,NaN,5.0,NaN,NaN,0.2,NaN
1999-05-01,NaN,-1.7,NaN,-2.3,-1.7,NaN,-1.65,NaN,-1.5,-3.99,...,-2.7,NaN,NaN,49.96,NaN,5.2,NaN,NaN,0.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,1.6,3.2,-1.3,-0.8,-12.6,NaN,-5.9,-8.4,-12.2,-8.4,...,1.0,4.3,0.1,49.41,NaN,-2.8,-8.3,-2.3,1.8,10.6
2023-09-01,2.2,2.0,-1.7,-1.0,-14.7,NaN,-4.6,-10.0,-14.1,-8.2,...,1.3,5.1,0.7,47.44,NaN,-2.8,-4.3,-2.5,0.8,16.7
2023-10-01,1.3,0.8,-1.2,-0.9,-11.0,4.1,-3.7,-7.8,-9.5,-8.1,...,0.8,5.8,1.3,39.39,-0.2,-4.1,-13.5,-6.6,0.2,21.6
2023-11-01,0.9,0.3,0.3,-1.3,-7.9,NaN,-4.2,NaN,-12.6,-7.7,...,0.6,4.6,1.2,42.25,NaN,-4.7,-8.9,37.0,0.8,21.9


In [13]:
preprocessor.process_monthly_indicator('Core CPI (MoM)')

1781 indicators read
Missing countries
['Japan', 'France', 'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark', 'Sweden', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'Taiwan', 'India', 'Korea', 'Brazil', 'Saudi Arabia', 'Thailand', 'Indonesia', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye', 'Philippines', 'Poland', 'Greece', 'Peru', 'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia']
Data coverage since 1999 (50 countries): 11.5 %
Data coverage since 2013 (30 countries): 14.5 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,United States,Core CPI (MoM),Core CPI (MoM) (Dec),1998-12-01 00:00:00,0.30
1998-12-01,1999-01-01 04:00:00,Canada,Core CPI (MoM),Core CPI (MoM) (Dec),1998-12-01 00:00:00,-0.20
1998-12-01,1999-01-01 04:00:00,Mexico,Core CPI (MoM),Core CPI (MoM) (Dec),1998-12-01 00:00:00,1.56
1998-12-01,1999-01-01 04:00:00,Eurozone,Core CPI (MoM),Core CPI (MoM) (Dec),1998-12-01 00:00:00,0.10
1998-12-01,1999-01-01 04:00:00,Chile,Core CPI (MoM),Core CPI (MoM) (Dec),1998-12-01 00:00:00,0.60
...,...,...,...,...,...,...
2023-11-01,2023-12-20 02:00:00,United Kingdom,Core CPI (MoM),Core CPI (MoM) (Nov),2023-11-01 00:00:00,-0.30
2023-12-01,2024-01-05 05:00:00,Eurozone,Core CPI (MoM),Core CPI (MoM),2023-12-01 00:00:00,0.40
2023-12-01,2024-01-08 06:00:00,Chile,Core CPI (MoM),Core CPI (MoM),2023-12-01 00:00:00,-0.40


In [14]:
preprocessor.process_monthly_indicator('Core CPI (YoY)')

3433 indicators read
Missing countries
['France', 'Switzerland', 'Germany', 'Australia', 'Netherlands', 'Denmark', 'Sweden', 'Hong Kong', 'Italy', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'Taiwan', 'India', 'Korea', 'Brazil', 'Saudi Arabia', 'Mexico', 'Indonesia', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye', 'Chile', 'Greece', 'Peru', 'Czechia', 'Colombia', 'Argentina', 'Russia']
Data coverage since 1999 (50 countries): 19.3 %
Data coverage since 2013 (30 countries): 19.6 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,Thailand,Core CPI (YoY),Core CPI (YoY) (Dec),1998-12-01 00:00:00,4.95
1998-12-01,1999-01-01 04:00:00,Hungary,Core CPI (YoY),Core CPI (YoY) (Dec),1998-12-01 00:00:00,11.50
1998-12-01,1999-01-01 04:00:00,United States,Core CPI (YoY),Core CPI (YoY) (Dec),1998-12-01 00:00:00,2.40
1998-12-01,1999-01-01 04:00:00,Canada,Core CPI (YoY),Core CPI (YoY) (Dec),1998-12-01 00:00:00,1.30
1998-12-01,1999-01-01 04:00:00,Japan,Core CPI (YoY),National Core CPI (YoY) (Dec),1998-12-01 00:00:00,-0.30
...,...,...,...,...,...,...
2023-12-01,2024-01-05 05:00:00,Eurozone,Core CPI (YoY),Core CPI (YoY),2023-12-01 00:00:00,3.40
2023-12-01,2024-01-10 11:00:00,Egypt,Core CPI (YoY),Core CPI (YoY) (Dec),2023-12-01 00:00:00,34.20
2023-12-01,2024-01-11 08:30:00,United States,Core CPI (YoY),Core CPI (YoY) (Dec),2023-12-01 00:00:00,3.90


## Trade

In [15]:
preprocessor.process_monthly_indicator('Trade Balance', 'Trade Balance (MoM)', pct_value=False)

13121 indicators read
Missing countries
['Belgium', 'Ireland', 'Saudi Arabia', 'Thailand', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Poland', 'Greece', 'Hungary', 'Egypt', 'Colombia']
Data coverage since 1999 (50 countries): 64.7 %
Data coverage since 2013 (30 countries): 81.4 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,United Kingdom,Trade Balance,Trade Balance (Dec),1998-12-01 00:00:00,-1.940000e+09
1998-12-01,1999-01-01 04:00:00,United States,Trade Balance,Trade Balance (Dec),1998-12-01 00:00:00,-1.470000e+10
1998-12-01,1999-01-01 04:00:00,Canada,Trade Balance,Trade Balance (Dec),1998-12-01 00:00:00,2.350000e+09
1998-12-01,1999-01-01 04:00:00,Mexico,Trade Balance,Trade Balance (Dec),1998-12-01 00:00:00,-8.100000e+08
1998-12-01,1999-01-01 04:00:00,Sweden,Trade Balance,Trade Balance (Dec),1998-12-01 00:00:00,1.140000e+10
...,...,...,...,...,...,...
2024-11-01,2024-01-12 00:30:00,Netherlands,Trade Balance,Dutch Trade Balance (Nov),2024-11-01 00:00:00,1.326000e+10
2024-11-01,2024-01-12 02:00:00,United Kingdom,Trade Balance,Trade Balance (Nov),2024-11-01 00:00:00,-1.419000e+10
2023-12-01,2024-01-14 06:00:00,Israel,Trade Balance,Trade Balance,2023-12-01 00:00:00,-2.029800e+09


In [16]:
preprocessor.process_monthly_indicator('Current Account', pct_value=False)

4384 indicators read
Missing countries
['Japan', 'Switzerland', 'Germany', 'Netherlands', 'Spain', 'Hong Kong', 'Italy', 'Singapore', 'Belgium', 'Israel', 'Ireland', 'New Zealand', 'Austria', 'China', 'Taiwan', 'India', 'Brazil', 'Saudi Arabia', 'Mexico', 'Thailand', 'Indonesia', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye', 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary', 'Egypt', 'Colombia', 'Argentina', 'Russia']
Data coverage since 1999 (50 countries): 10.5 %
Data coverage since 2013 (30 countries): 13.7 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,Korea,Current Account,Current Account (Dec),1998-12-01 00:00:00,3.260000e+09
1999-01-01,1999-02-01 04:00:00,Korea,Current Account,Current Account (Jan),1999-01-01 00:00:00,2.550000e+09
1999-01-01,1999-02-01 04:00:00,Eurozone,Current Account,Current Account (Jan),1999-01-01 00:00:00,-1.590000e+10
1999-02-01,1999-03-01 04:00:00,Korea,Current Account,Current Account (Feb),1999-02-01 00:00:00,2.570000e+09
1999-03-01,1999-04-01 04:00:00,Korea,Current Account,Current Account (Mar),1999-03-01 00:00:00,1.900000e+09
...,...,...,...,...,...,...
2024-11-01,2024-01-08 18:00:00,Korea,Current Account,Current Account (Nov),2024-11-01 00:00:00,4.060000e+09
2024-11-01,2024-01-09 02:00:00,Denmark,Current Account,Current Account (Nov),2024-11-01 00:00:00,3.050000e+10
2024-11-01,2024-01-09 02:45:00,France,Current Account,French Current Account (Nov),2024-11-01 00:00:00,-2.800000e+09


## Bussiness

In [17]:
manufacturing_pmi = preprocessor.process_pmi_indicator('Manufacturing PMI', 'procure.ch PMI')
manufacturing_pmi

6737 indicators read
Missing countries
['Netherlands', 'Singapore', 'Finland', 'Belgium', 'Israel', 'New Zealand', 'Portugal', 'Saudi Arabia', 'Thailand', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Chile', 'Peru', 'Czechia', 'Egypt', 'Colombia', 'Argentina']
Data coverage since 1999 (50 countries): 31.9 %
Data coverage since 2013 (30 countries): 77.6 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,Sweden,Manufacturing PMI,Manufacturing PMI (Dec),1998-12-01,44.1
1998-12-01,1999-01-01 04:00:00,Switzerland,Manufacturing PMI,procure.ch PMI (Dec),1998-12-01,51.4
1998-12-01,1999-01-01 04:00:00,Hungary,Manufacturing PMI,Manufacturing PMI (Dec),1998-12-01,47.7
1999-01-01,1999-02-01 04:00:00,Hungary,Manufacturing PMI,Manufacturing PMI (Jan),1999-01-01,57.3
1999-01-01,1999-02-01 04:00:00,Sweden,Manufacturing PMI,Manufacturing PMI (Jan),1999-01-01,48.0
...,...,...,...,...,...,...
2023-12-01,2024-01-03 00:00:00,India,Manufacturing PMI,S&P Global India Manufacturing PMI (Dec),2023-12-01,54.9
2023-12-01,2024-01-03 03:30:00,Switzerland,Manufacturing PMI,procure.ch PMI (Dec),2023-12-01,43.0
2023-12-01,2024-01-03 19:30:00,Japan,Manufacturing PMI,au Jibun Bank Japan Manufacturing PMI (Dec),2023-12-01,47.9


In [18]:
df = preprocessor.transform_to_countries_df(manufacturing_pmi, freq='M')
df.dropna(thresh=5, axis=1)

,United States,Japan,United Kingdom,Canada,France,Switzerland,Germany,Australia,Denmark,Sweden,...,Brazil,South Africa,Mexico,Indonesia,Turkiye,Philippines,Poland,Greece,Hungary,Russia
1999-01-01,NaN,NaN,NaN,NaN,NaN,46.5,NaN,NaN,NaN,48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.3,NaN
1999-02-01,NaN,NaN,NaN,NaN,NaN,48.0,NaN,NaN,NaN,48.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56.7,NaN
1999-03-01,NaN,NaN,NaN,NaN,NaN,48.7,NaN,NaN,NaN,51.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.4,NaN
1999-04-01,NaN,NaN,NaN,NaN,NaN,51.2,NaN,NaN,NaN,53.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.5,NaN
1999-05-01,NaN,NaN,NaN,NaN,NaN,51.8,NaN,NaN,NaN,51.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,47.9,49.6,43.0,48.0,46.0,39.9,39.1,49.6,NaN,45.8,...,50.1,49.7,51.2,53.9,49.0,49.7,43.1,52.9,46.5,52.7
2023-09-01,49.8,48.5,44.3,47.5,44.2,44.9,39.6,48.7,NaN,43.3,...,49.0,45.4,49.8,52.3,49.6,50.6,43.9,50.3,47.4,54.5
2023-10-01,50.0,48.7,44.8,48.6,42.8,40.6,40.8,48.2,NaN,45.7,...,48.6,45.4,52.1,51.5,48.4,52.4,44.5,50.8,50.5,53.8
2023-11-01,49.4,48.3,47.2,47.7,42.9,42.1,42.6,47.7,NaN,49.0,...,49.4,48.2,52.5,51.7,47.2,52.7,48.7,50.9,52.2,53.8


In [19]:
services_pmi = preprocessor.process_pmi_indicator('Services PMI')
services_pmi

3005 indicators read
Missing countries
['Canada', 'Switzerland', 'Netherlands', 'Denmark', 'Hong Kong', 'Singapore', 'Finland', 'Belgium', 'Norway', 'Israel', 'New Zealand', 'Austria', 'Portugal', 'China', 'Taiwan', 'Korea', 'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye', 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina']
Data coverage since 1999 (50 countries): 13.8 %
Data coverage since 2013 (30 countries): 39.5 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
2006-01-01,2006-02-01 04:00:00,Sweden,Services PMI,Services PMI (MoM) (Jan),2006-01-01,59.0
2006-02-01,2006-03-01 04:00:00,Sweden,Services PMI,Services PMI (MoM) (Feb),2006-02-01,58.5
2006-03-01,2006-04-01 04:00:00,Sweden,Services PMI,Services PMI (MoM) (Mar),2006-03-01,62.2
2006-04-01,2006-05-01 04:00:00,Sweden,Services PMI,Services PMI (MoM) (Apr),2006-04-01,56.8
2006-05-01,2006-06-01 04:00:00,Sweden,Services PMI,Services PMI (MoM) (May),2006-05-01,60.5
...,...,...,...,...,...,...
2023-12-01,2024-01-04 04:30:00,United Kingdom,Services PMI,S&P Global/CIPS UK Services PMI (Dec),2023-12-01,53.4
2023-12-01,2024-01-04 08:00:00,Brazil,Services PMI,S&P Global Services PMI (Dec),2023-12-01,50.5
2023-12-01,2024-01-04 09:45:00,United States,Services PMI,S&P Global Services PMI (Dec),2023-12-01,51.4


In [20]:
df = preprocessor.transform_to_countries_df(services_pmi, freq='M')
df.dropna(thresh=5, axis=1)

,United States,Japan,United Kingdom,France,Germany,Australia,Sweden,Spain,Italy,Ireland,Eurozone,India,Brazil,Russia
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,50.5,54.3,49.5,46.0,47.3,47.8,49.0,49.3,49.8,55.0,47.9,60.1,50.6,57.6
2023-09-01,50.1,53.8,49.3,44.4,50.3,51.8,46.3,50.5,49.9,54.5,48.7,61.0,48.7,55.4
2023-10-01,50.6,51.6,49.5,45.2,48.2,47.9,48.5,51.1,47.7,52.6,47.8,58.4,51.0,53.6
2023-11-01,50.8,50.8,50.9,45.4,49.6,46.0,48.3,51.0,49.5,54.2,48.7,56.9,51.2,52.2


In [21]:
preprocessor.process_pmi_indicator('Composite PMI')

1559 indicators read
Missing countries
['Japan', 'Canada', 'Switzerland', 'Australia', 'Netherlands', 'Denmark', 'Sweden', 'Spain', 'Hong Kong', 'Singapore', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria', 'Portugal', 'Taiwan', 'India', 'Korea', 'South Africa', 'Mexico', 'Thailand', 'Indonesia', 'Malaysia', 'Qatar', 'Kuwait', 'Turkiye', 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia']
Data coverage since 1999 (50 countries): 7.3 %
Data coverage since 2013 (30 countries): 19.8 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
2012-03-01,2012-04-04 02:48:00,France,Composite PMI,HCOB France Composite PMI,2012-03-01,48.7
2012-04-01,2012-05-04 02:48:00,France,Composite PMI,HCOB France Composite PMI,2012-04-01,45.9
2012-05-01,2012-06-05 02:48:00,France,Composite PMI,HCOB France Composite PMI,2012-05-01,44.6
2012-06-01,2012-07-01 04:00:00,United States,Composite PMI,S&P Global Composite PMI (Jun),2012-06-01,53.2
2012-06-01,2012-07-01 04:00:00,Germany,Composite PMI,HCOB Germany Composite PMI (Jun),2012-06-01,48.1
...,...,...,...,...,...,...
2023-12-01,2024-01-04 03:55:00,Germany,Composite PMI,HCOB Germany Composite PMI (Dec),2023-12-01,47.4
2023-12-01,2024-01-04 04:00:00,Eurozone,Composite PMI,HCOB Eurozone Composite PMI (Dec),2023-12-01,47.6
2023-12-01,2024-01-04 04:30:00,United Kingdom,Composite PMI,S&P Global/CIPS UK Composite PMI (Dec),2023-12-01,52.1


In [22]:
preprocessor.process_pmi_indicator('Construction PMI')

291 indicators read
Missing countries
['United States', 'Japan', 'Canada', 'Switzerland', 'Australia', 'Netherlands', 'Denmark', 'Sweden', 'Spain', 'Hong Kong', 'Singapore', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'Taiwan', 'India', 'Korea', 'Brazil', 'Saudi Arabia', 'South Africa', 'Mexico', 'Thailand', 'Indonesia', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Turkiye', 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia']
Data coverage since 1999 (50 countries): 1.9 %
Data coverage since 2013 (30 countries): 5.4 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
2008-02-01,2008-03-04 04:30:00,United Kingdom,Construction PMI,S&P Global / CIPS UK Construction PMI,2008-02-01,52.4
2008-03-01,2008-04-02 03:30:00,United Kingdom,Construction PMI,S&P Global / CIPS UK Construction PMI,2008-03-01,47.2
2008-04-01,2008-05-02 03:30:00,United Kingdom,Construction PMI,S&P Global / CIPS UK Construction PMI,2008-04-01,46.1
2008-05-01,2008-06-03 04:30:00,United Kingdom,Construction PMI,S&P Global / CIPS UK Construction PMI,2008-05-01,43.9
2008-06-01,2008-07-02 03:30:00,United Kingdom,Construction PMI,S&P Global / CIPS UK Construction PMI,2008-06-01,38.8
...,...,...,...,...,...,...
2023-12-01,2024-01-05 03:30:00,Italy,Construction PMI,HCOB Italy Construction PMI (MoM) (Dec),2023-12-01,55.2
2023-12-01,2024-01-05 03:30:00,Germany,Construction PMI,HCOB Germany Construction PMI (Dec),2023-12-01,37.0
2023-12-01,2024-01-05 03:30:00,France,Construction PMI,HCOB France Construction PMI (MoM) (Dec),2023-12-01,42.6


In [23]:
preprocessor.process_monthly_indicator('Industrial Production (MoM)', 'Industrial Output (MoM)')

4363 indicators read
Missing countries
['Canada', 'Switzerland', 'Australia', 'Netherlands', 'Spain', 'Hong Kong', 'Finland', 'Belgium', 'Norway', 'Ireland', 'New Zealand', 'Austria', 'Portugal', 'China', 'Taiwan', 'India', 'Saudi Arabia', 'South Africa', 'Thailand', 'Indonesia', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia']
Data coverage since 1999 (50 countries): 27.6 %
Data coverage since 2013 (30 countries): 36.1 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,United Kingdom,Industrial Production (MoM),Industrial Production (MoM) (Dec),1998-12-01 00:00:00,0.30
1998-12-01,1999-01-01 04:00:00,United States,Industrial Production (MoM),Industrial Production (MoM) (Dec),1998-12-01 00:00:00,0.40
1998-12-01,1999-01-01 04:00:00,Mexico,Industrial Production (MoM),Industrial Production (MoM) (Dec),1998-12-01 00:00:00,0.67
1998-12-01,1999-01-01 04:00:00,Italy,Industrial Production (MoM),Italian Industrial Production (MoM) (Dec),1998-12-01 00:00:00,-2.30
1998-12-01,1999-01-01 04:00:00,Korea,Industrial Production (MoM),Industrial Production (MoM) (Dec),1998-12-01 00:00:00,2.30
...,...,...,...,...,...,...
2024-11-01,2024-01-10 02:00:00,Turkiye,Industrial Production (MoM),Industrial Production (MoM) (Nov),2024-11-01 00:00:00,-2.80
2024-11-01,2024-01-10 02:45:00,France,Industrial Production (MoM),French Industrial Production (MoM) (Nov),2024-11-01 00:00:00,0.50
2024-11-01,2024-01-11 04:00:00,Italy,Industrial Production (MoM),Italian Industrial Production (MoM) (Nov),2024-11-01 00:00:00,-1.50


In [24]:
preprocessor.process_monthly_indicator('Industrial Production (YoY)', 'Industrial Output (YoY)')

6570 indicators read
Missing countries
['Japan', 'Canada', 'France', 'Australia', 'Netherlands', 'Denmark', 'Finland', 'Belgium', 'Norway', 'Israel', 'Ireland', 'New Zealand', 'Portugal', 'Saudi Arabia', 'South Africa', 'Indonesia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Chile', 'Peru', 'Egypt']
Data coverage since 1999 (50 countries): 42.3 %
Data coverage since 2013 (30 countries): 50.1 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,United Kingdom,Industrial Production (YoY),Industrial Production (YoY) (Dec),1998-12-01 00:00:00,0.40
1998-12-01,1999-01-01 04:00:00,Mexico,Industrial Production (YoY),Industrial Production (YoY) (Dec),1998-12-01 00:00:00,3.65
1998-12-01,1999-01-01 04:00:00,Italy,Industrial Production (YoY),Italian Industrial Production (YoY) (Dec),1998-12-01 00:00:00,-6.00
1998-12-01,1999-01-01 04:00:00,Korea,Industrial Production (YoY),Industrial Production (YoY) (Dec),1998-12-01 00:00:00,6.60
1998-12-01,1999-01-01 04:00:00,Spain,Industrial Production (YoY),Spanish Industrial Production (YoY) (Dec),1998-12-01 00:00:00,5.20
...,...,...,...,...,...,...
2024-11-01,2024-01-11 04:00:00,Italy,Industrial Production (YoY),Italian Industrial Production (YoY) (Nov),2024-11-01 00:00:00,-3.10
2024-11-01,2024-01-11 07:00:00,Mexico,Industrial Production (YoY),Industrial Production (YoY) (Nov),2024-11-01 00:00:00,2.80
2024-11-01,2024-01-12 02:00:00,United Kingdom,Industrial Production (YoY),Industrial Production (YoY) (Nov),2024-11-01 00:00:00,-0.10


## Consumer

In [25]:
preprocessor.process_monthly_indicator('Retail Sales (MoM)')

5150 indicators read
Missing countries
['Japan', 'France', 'Switzerland', 'Netherlands', 'Denmark', 'Spain', 'Hong Kong', 'Finland', 'Belgium', 'Norway', 'Israel', 'Austria', 'Portugal', 'China', 'Taiwan', 'India', 'Saudi Arabia', 'South Africa', 'Thailand', 'Indonesia', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia']
Data coverage since 1999 (50 countries): 25.6 %
Data coverage since 2013 (30 countries): 37.2 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,United Kingdom,Retail Sales (MoM),Retail Sales (MoM) (Dec),1998-12-01 00:00:00,-0.7
1998-12-01,1999-01-01 04:00:00,United States,Retail Sales (MoM),Retail Sales (MoM) (Dec),1998-12-01 00:00:00,0.9
1998-12-01,1999-01-01 04:00:00,Canada,Retail Sales (MoM),Retail Sales (MoM) (Dec),1998-12-01 00:00:00,-0.5
1998-12-01,1999-01-01 04:00:00,Sweden,Retail Sales (MoM),Retail Sales (MoM) (Dec),1998-12-01 00:00:00,-1.0
1998-12-01,1999-01-01 04:00:00,Germany,Retail Sales (MoM),German Retail Sales (MoM) (Dec),1998-12-01 00:00:00,-2.2
...,...,...,...,...,...,...
2024-11-01,2024-01-08 19:30:00,Australia,Retail Sales (MoM),Retail Sales (MoM) (Nov),2024-11-01 00:00:00,2.0
2024-11-01,2024-01-09 06:00:00,Ireland,Retail Sales (MoM),Irish Retail Sales (MoM) (Nov),2024-11-01 00:00:00,0.2
2024-11-01,2024-01-10 02:00:00,Sweden,Retail Sales (MoM),Retail Sales (MoM) (Nov),2024-11-01 00:00:00,-0.5


In [26]:
preprocessor.process_monthly_indicator('Retail Sales (YoY)')

7530 indicators read
Missing countries
['Canada', 'France', 'Australia', 'Finland', 'Belgium', 'Norway', 'Israel', 'Austria', 'Portugal', 'Taiwan', 'India', 'Korea', 'Saudi Arabia', 'Thailand', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Philippines', 'Peru', 'Egypt']
Data coverage since 1999 (50 countries): 46.8 %
Data coverage since 2013 (30 countries): 55.8 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,United Kingdom,Retail Sales (YoY),Retail Sales (YoY) (Dec),1998-12-01 00:00:00,-0.30
1998-12-01,1999-01-01 04:00:00,Sweden,Retail Sales (YoY),Retail Sales (YoY) (Dec),1998-12-01 00:00:00,3.70
1998-12-01,1999-01-01 04:00:00,Germany,Retail Sales (YoY),German Retail Sales (YoY) (Dec),1998-12-01 00:00:00,2.10
1998-12-01,1999-01-01 04:00:00,Netherlands,Retail Sales (YoY),Dutch Retail Sales (YoY) (Dec),1998-12-01 00:00:00,7.84
1998-12-01,1999-01-01 04:00:00,Japan,Retail Sales (YoY),Retail Sales (YoY) (Dec),1998-12-01 00:00:00,0.70
...,...,...,...,...,...,...
2024-11-01,2024-01-09 20:00:00,Indonesia,Retail Sales (YoY),Retail Sales (YoY) (Nov),2024-11-01 00:00:00,2.10
2024-11-01,2024-01-10 02:00:00,Sweden,Retail Sales (YoY),Retail Sales (YoY) (Nov),2024-11-01 00:00:00,-1.70
2024-11-01,2024-01-10 04:00:00,Italy,Retail Sales (YoY),Italian Retail Sales (YoY) (Nov),2024-11-01 00:00:00,1.50


In [27]:
preprocessor.process_monthly_indicator('Consumer Confidence', 'Consumer confidence')

6112 indicators read
Missing countries
['Japan', 'Canada', 'Switzerland', 'Germany', 'Australia', 'Hong Kong', 'Singapore', 'Israel', 'New Zealand', 'Austria', 'China', 'Taiwan', 'India', 'Saudi Arabia', 'South Africa', 'Malaysia', 'United Arab Emirates', 'Qatar', 'Kuwait', 'Philippines', 'Poland', 'Chile', 'Greece', 'Peru', 'Hungary', 'Czechia', 'Egypt', 'Colombia', 'Argentina', 'Russia']
Data coverage since 1999 (50 countries): 33.9 %
Data coverage since 2013 (30 countries): 44.5 %


,ReportDateTime,Country,Indicator,FullName,Period,Value
Period,,,,,,
1998-12-01,1999-01-01 04:00:00,United Kingdom,Consumer Confidence,GfK Consumer Confidence (Dec),1998-12-01 00:00:00,-8.0
1998-12-01,1999-01-01 04:00:00,United States,Consumer Confidence,CB Consumer Confidence (Dec),1998-12-01 00:00:00,126.7
1998-12-01,1999-01-01 04:00:00,Sweden,Consumer Confidence,Consumer Confidence (Dec),1998-12-01 00:00:00,101.0
1998-12-01,1999-01-01 04:00:00,Italy,Consumer Confidence,Italian Consumer Confidence (Dec),1998-12-01 00:00:00,111.2
1998-12-01,1999-01-01 04:00:00,Netherlands,Consumer Confidence,Dutch Consumer Confidence (Dec),1998-12-01 00:00:00,9.0
...,...,...,...,...,...,...
2023-12-01,2024-01-02 06:00:00,Portugal,Consumer Confidence,Portuguese Consumer Confidence (Dec),2023-12-01 00:00:00,-28.2
2023-12-01,2024-01-08 05:00:00,Eurozone,Consumer Confidence,Consumer Confidence,2023-12-01 00:00:00,-15.0
2023-12-01,2024-01-08 07:00:00,Mexico,Consumer Confidence,Consumer Confidence,2023-12-01 00:00:00,46.8
